# Plot azimuthal heatmaps for selected regions of interest

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
#To import DanMAX from the folder above:
import sys
sys.path.append('../')
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

#### import data

In [ ]:
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/**/scan-####.h5'
fname = DM.findScan()
aname = DM.getAzintFname(fname)

# read azimuthally binned data
data = DM.getAzintData(aname)

# determine and read radial unit
if type(data['q']) != type(None):
    x = data['q']
    x_edge = data['q_edge']
    Q = True
else:
    x = data['tth']
    x_edge = data['tth_edge']
    Q = False
# read common meta data from the master file
meta = DM.getMetaData(fname)

# reduce time-resolution to speed up initial analysis
rf = 1
start = None 
end =  None
data = DM.reduceDic(data,reduction_factor=rf,start=start,end=end)
meta = DM.reduceDic(meta,reduction_factor=rf,start=start,end=end)

I = data['I']
cake = data['cake'][:,:,:] # shape : [frames, azi bins, radial bins]
cake[cake<=0]=np.nan
azi = data['azi'][:]
azi_edge = data['azi_edge'][:]

t = meta['time'] # relative time stamp in seconds
T = meta['temp'] # temperature in Kelvin (if available, otherwise None)
I0 = meta['I0']  # relative incident beam intensity "I zero"
E = meta['energy'] # X-ray energy in keV

# normalize to I0
I = (I.T/I0).T
cake = (cake.T/I0).T

I_avg = np.mean(I,axis=0)

# "time edge"
t_edge = np.append(t,t[-1]+np.mean(np.diff(t)))

print(f'Effective time-resolution: {np.mean(np.diff(t)):.2f} s')

#### Estimate peak positions and regions of interest
Try to guess peaks and peak positions and output a simple list for copy-pasting (Tip: press down `alt` to enable columnwise cursor selection)  
It might be necessary to tweak the `find_peaks` *prominence* and *wlen* parameters

In [ ]:
peaks, prop = find_peaks((I_avg-I_avg.min())/(I_avg.max()-I_avg.min())*100,
                         prominence=0.5,
                         wlen=100)
# correct for overlap
prop['right_bases'][:-1] = np.array([min(prop['left_bases'][i+1],prop['right_bases'][i]) for i in range(len(peaks)-1)])

roi_bgr = x != x
# initialize figure
fig = plt.figure()
plt.plot(x, I_avg,'k.-',ms=1.5,lw=1,label='average pattern')
print(' #  hkl  :    ROI (°)')
for i,peak in enumerate(peaks):
    roi = x[prop['left_bases'][i]],x[prop['right_bases'][i]]
    print(f"{i+1:>2d} '?{i+1}?' : [{roi[0]:>5.2f}, {roi[1]:>5.2f}],")
    roi = (x>=roi[0]) & (x<roi[1])
    roi_bgr += roi # region of interest for background points (inverted)
    # plot average diffraction pattern and heatmap
    color = plt.plot(x[roi], I_avg[roi],'.',ms=3,label=i)[-1].get_color()
    plt.annotate(f'#{i+1}',(x[peak],I_avg[peak]+1),color=color)
    
roi_bgr = ~roi_bgr    
plt.xlabel(r'$2\theta (°)$')
plt.yticks([])
plt.yscale('log')

#### Select ROIs
Copy-paste the desired ROI from the output of the previous cell.  

In [ ]:
#              label  :   ROI         
reflections = {'hkl' : [ 8.04,  9.21],
              }

bgr_endpoints = [5.,35.]

# update background points
roi = (x<=bgr_endpoints[0]) ^ (x>bgr_endpoints[1])
roi_bgr[roi]=False

# initialize figure
fig = plt.figure()

plt.plot(x, np.nanmean(cake,axis=(0,1)),'k-',label='average pattern')
for hkl in reflections:
    roi = reflections[hkl]
    roi = (x>roi[0]) & (x<roi[1])
    # plot average diffraction pattern and heatmap
    plt.plot(x[roi], I_avg[roi],'.',ms=3,label=hkl)
plt.plot(x[roi_bgr], I_avg[roi_bgr],'.',c='grey',ms=1.5,label='bgr points')
plt.legend()
if Q:
    plt.xlabel(r'$Q (A^{-1})$')
else:
    plt.xlabel(r'$2\theta (°)$')
plt.yticks([])
#plt.yscale('log')

In [ ]:
subtract_background = False
chebyshev_bgr = False
use_mean = False # toggle whether to use mean rather than integral


if chebyshev_bgr:
    ## Chebyshev background polynomial degree
    deg = 5
    # background intensity averaged over the azimuthal angle
    I_bgr = np.nanmean(cake[:,:,roi_bgr],axis=1)
    # crop nan-values
    x_bgr = x[roi_bgr][~np.isnan(np.sum(I_bgr,axis=0))]
    I_bgr = I_bgr[:,~np.isnan(np.sum(I_bgr,axis=0))]
    # fit polynomial
    coef = np.polynomial.chebyshev.chebfit(x_bgr, I_bgr.T, deg)
    # evaluate fit at x
    y_bgr = np.polynomial.chebyshev.chebval(x,coef)
    # ensure background subtraction is non-negative
    y_bgr += np.nanmin(cake.transpose(1,0,2)-y_bgr)

# Set the number of columns for the figure
cols = 3

rows = int(len(reflections)/cols) + (len(reflections)%cols!=0)

# initialize figure
fig, axes = plt.subplots(rows,cols,sharex=True)
fig.set_size_inches(12,8)
fig.suptitle(DM.getScan_id(fname))
axes = axes.flatten()
for ax in axes:
    #ax.set_xticks()
    #ax.set_yticks()
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Azimuthal angle (deg)')

pcm = [] # pcolormeshes
for k,hkl in enumerate(reflections):

    roi = (x>reflections[hkl][0]) & (x<reflections[hkl][1])

    # Modification to np.trapz to better handle nan values
    # nan values are (assumed) invariant along the sample rotation axis (omega)
    # and only change along the azimuthal and radial axes
    # shape: [time,azi,radial]
    y = np.full(cake.shape[:2],np.nan)
    # loop through the azimuthal axis
    for j in range(cake.shape[1]):
        yi = cake[:,j,roi]
        if subtract_background:
            if chebyshev_bgr:
                if use_mean:
                    bgr = y_bgr[:,roi].T
                else:
                    bgr = y_bgr[:,roi][:,~np.isnan(yi[0])].T
            else:
                bgr = (np.nanmean(yi[:,:3],axis=1)+np.nanmean(yi[:,-3:],axis=1))/2
        else:
            bgr = 0
        if use_mean:
            y[:,j] = np.mean((yi.T-bgr).T,axis=1)
        # if a sufficient number of points are non-nan, estimate the integral of the peaks along the 2theta axis
        elif np.count_nonzero(np.isnan(yi[0]))<(len(yi[0])*0.1):
            y[:,j] = np.trapz((yi[:,~np.isnan(yi[0])].T- bgr).T,
                              x=x[roi][~np.isnan(yi[0])],
                              axis=1)
    # set non-physical (negative) values to nan  
    y[y<0]=np.nan
    y -= np.nanmin(y)
    roi = (x_edge>reflections[hkl][0]) & (x_edge<reflections[hkl][1])
    
    if k>0:
        #print(vmin,vmax)
        y /= y_max/100
        vmin,vmax = min(vmin,np.nanmin(y)), max(vmax,np.nanmax(y))
        #print(vmin,vmax)
    else:
        y_max = np.nanmax(y)
        y /= y_max/100
        vmin,vmax = np.nanmin(y) ,np.nanmax(y)
    
    # plot heatmap
    ax = axes[k]
    ax.set_title(f'({hkl})')
    ax.grid(False)
    pcm.append(ax.pcolormesh(t_edge,
                              azi_edge,
                              y.T,
                              norm='linear'))

share_clim = False
if share_clim:
    for p in pcm:
        p.set_clim(vmin,vmax)
else:
    for i,p in enumerate(pcm):
        fig.colorbar(p,
                     ax=axes[i],
                     fraction=0.05,
                     shrink=0.75)
#fig.colorbar(p,label='a.u.')

# delete surplus plots
for i in range(1,cols*rows-len(reflections)+1):
    fig.delaxes(axes[-i])
fig.tight_layout()